In this notebook, we'll revisit our text classification example, but will use a one dimensional CNN this time around.

In [1]:
%tensorflow_version 2.x

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, Dropout, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import sequence
import numpy as np

These steps are the same as before:

In [2]:
max_features   = 20000
maxlen         = 80
embedding_size = 128

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test),  'test sequences')

17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [4]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test  = sequence.pad_sequences(X_test,  maxlen=maxlen)

print('X_train shape:', X_train.shape)
print('X_test shape:',  X_test.shape)

X_train shape: (25000, 80)
X_test shape: (25000, 80)


Next, we define the network:

In [5]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.2))

model.add(Conv1D(128, 3, activation='relu', strides=1, padding="same"))
model.add(GlobalMaxPooling1D())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 128)           2560000   
_________________________________________________________________
dropout (Dropout)            (None, 80, 128)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 80, 128)           49280     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

And fit for three epochs:

In [6]:
model.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test, batch_size=128)
print('Test score:   ', score)
print('Test accuracy:', acc)

Epoch 1/3
196/196 [==============================] - 7s 38ms/step - loss: 0.5220 - accuracy: 0.7239 - val_loss: 0.3575 - val_accuracy: 0.8408
Epoch 2/3
196/196 [==============================] - 7s 36ms/step - loss: 0.2713 - accuracy: 0.8908 - val_loss: 0.3322 - val_accuracy: 0.8555
Epoch 3/3
196/196 [==============================] - 1s 4ms/step - loss: 0.3907 - accuracy: 0.8473
Test score:    0.3907119631767273
Test accuracy: 0.847320020198822


Comparing this result with the RNN approaches tried earlier, it seems that a one dimensional CNN does even better in this case!